# Elasticidad de la demanda

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import statsmodels.api as sm

from src.dynamic_pricing_data_loader import cargar_y_preparar_datos

In [2]:
def Get_goodDay(df,atribute,atribute1):
    
    df_output=df.copy()
    df_output['FECHA_CORRIDA'] = pd.to_datetime(df_output['FECHA_CORRIDA'])
    df_output['Dia de la semana'] = df_output[atribute1].dt.day_name()
    # Calculate the sum of 'VENTA_BOLETOS' for each day of the week
    daily_sales = df_output.groupby('Dia de la semana')[atribute].sum()
    daily_sales=daily_sales.sort_values(ascending=False)
    daily_sales=daily_sales[:3]
    indices={'Monday':0,'Tuesday':1, 'Wednesday':2, 'Thursday':3,
         'Friday':4,'Saturday':5,'Sunday':6}
    dias=list(daily_sales.index.map(indices).to_numpy())
    df_output['Buen_dia'] =df_output[atribute1].dt.dayofweek.isin(dias).astype(int)
    df_output['Buen_dia']= df_output['Buen_dia'].astype(np.int64)
    df_output.drop('Dia de la semana', axis=1, inplace=True)
    
    return df_output

In [3]:
def GetData(year): 
    # Definir rutas
    ruta_principal = os.getcwd()
    config_path = os.path.join(ruta_principal, "config", "config.json")
    
    # Cargar y preparar datos
    Frame= cargar_y_preparar_datos(config_path, ruta_principal)
    Df=Frame[['OPERACION','FECHA_CORRIDA','FECHA_OPERACION','TIPO_PASAJERO','VENTA',
                 'KMS_TRAMO','PAX_SUBEN','VENTA_ANTICIPADA','TIPO_CORRIDA',
                 'DESC_DESCUENTO','PORCENT_PROMO','IVA_VENDIDO','TARIFA_BASE_TRAMO','IVA_TARIFA_BASE_TRAMO']]

    Df=Df[Df['VENTA']>0]
    #Df.to_excel("NuevaBase.xlsx", index=False)
    Df['FECHA_OPERACION'] = pd.to_datetime(Df['FECHA_OPERACION'])
    Df = Df[Df['FECHA_OPERACION'].dt.year == year]
    Df['PRECIO'] = Df['VENTA']-Df['IVA_VENDIDO']
    Df['TARIFA'] = Df['TARIFA_BASE_TRAMO']-Df['IVA_TARIFA_BASE_TRAMO']
    Df=Df[['OPERACION','FECHA_CORRIDA','FECHA_OPERACION','TIPO_PASAJERO','VENTA','PRECIO',
                     'KMS_TRAMO','PAX_SUBEN','VENTA_ANTICIPADA','TIPO_CORRIDA','TARIFA',
                     'DESC_DESCUENTO','PORCENT_PROMO']]
    
    Df['PORCENT_PROMO'] = Df['PORCENT_PROMO'].fillna(0)
    return Df[Df['TIPO_CORRIDA']=='NORMAL']

In [4]:
def GetFeatures(df):
    Frame=df.copy()
    Listado_Pasajeros=list(set(Frame['TIPO_PASAJERO']))
    #Frame=Frame[Frame['TIPO_PASAJERO']==Listado_Pasajeros[2]]
    # Convierte las columnas a tipo datetime
    Frame['FECHA_OPERACION'] = pd.to_datetime(Frame['FECHA_OPERACION'])
    Frame['FECHA_CORRIDA'] = pd.to_datetime(Frame['FECHA_CORRIDA'])
    
    # Crea una nueva columna 'DIAS_DIFERENCIA' con el cálculo
    Frame['Dias_Anticipacion'] = (Frame['FECHA_CORRIDA'] - Frame['FECHA_OPERACION']).dt.days
    Frame['Mes_Viaje'] = Frame['FECHA_CORRIDA'].dt.month
    Frame['Fin_Semana_Viaje'] = Frame['FECHA_CORRIDA'].dt.dayofweek >= 5
    Frame.rename(columns={'PAX_SUBEN': 'Q_Boletos'}, inplace=True)
    
    Frame=Get_goodDay(Frame,'PRECIO','FECHA_CORRIDA')
    Frame['Fin_Semana_Viaje'] = Frame['Fin_Semana_Viaje'].astype(int)
    Frame['Buen_dia'] = Frame['Buen_dia'].astype(int)
    Frame=Frame.dropna()
    # Cuenta el número de NaN por cada columna
    nan_por_columna = Frame.isnull().sum()
    
    #print("Cantidad de NaN por columna:")
    #print(nan_por_columna)
    return Frame

In [5]:
def GetElasticity(df):
    Frame=df.copy()
    X_P=Frame[['TARIFA', 'Dias_Anticipacion', 'Mes_Viaje', 'Fin_Semana_Viaje', 'Buen_dia']]
    Y_P=pd.DataFrame(Frame['PRECIO'])
    
    X_Q=Frame[['Dias_Anticipacion', 'Mes_Viaje', 'Fin_Semana_Viaje', 'Buen_dia']]
    
    X_P= sm.add_constant(X_P)  # Agregar una constante para el intercepto
    
    # Entrenar el modelo de regresión OLS para la primera etapa
    reg_stage1 = sm.OLS(Y_P, X_P).fit()
    
    # Ver el resumen de la segunda regresión
    #print(reg_stage1.summary())
    
    # Obtener los valores predichos del precio de venta
    T_Est = reg_stage1.predict(X_P)
    
    T_Est = pd.DataFrame(T_Est, columns=['T_Est'])
    
    X_Q = pd.concat([T_Est, X_Q], axis=1)
    X_Q = sm.add_constant(X_Q) # Agregar una constante para el intercepto
    Y_Q=pd.DataFrame(Frame['Q_Boletos'])
    
    # Entrenar el modelo de regresión OLS para la primera etapa
    reg_stage2 = sm.OLS(Y_Q, X_Q).fit()
    
    # Ver el resumen de la segunda regresión
    #print(reg_stage2.summary())
    
    # Obtener el coeficiente beta1 del resumen
    beta1 = reg_stage2.params['T_Est']
    
    # La variable endógena (P) es P_Venta
    P =pd.DataFrame(Frame['PRECIO'])
    # La variable dependiente (Y) es Q_Boletos
    Y =pd.DataFrame(Frame['Q_Boletos'])
    
    # Calcular los promedios de las variables para el cálculo de la elasticidad
    promedio_Q_Boletos = Y.mean().values[0]
    promedio_P_Venta = P.mean().values[0]
    
    # Calcular la elasticidad de la demanda
    elasticidad = beta1 * (promedio_P_Venta / promedio_Q_Boletos)
    
    print(f"El coeficiente beta1 es: {beta1:.4f}")
    print(f"La elasticidad de la demanda es: {elasticidad:.4f}")
    
    Coef = list(reg_stage2.params)
    return Coef, elasticidad

In [6]:
def GetPrizes(Coef,CondIni,UT):
    PrecioMaximo= Coef[0]+Coef[2]*CondIni['Dias_Anticipacion']+\
        +Coef[3]*CondIni['Mes_Viaje']+Coef[4]*CondIni['Fin_Semana_Viaje']+\
        +Coef[5]*CondIni[ 'Buen_dia']
        
    PM= np.abs(PrecioMaximo/(-2*Coef[1]))
    PS= np.abs(Coef[0]/(Coef[1]*2))
    Dp= (UT-PM)/UT
    PrecioMaximo= Dp*PM + PM
    PrecioSugerido = Dp*PS + PS
    print(f"El precio maximo a vender es: {PrecioMaximo:.4f}")
    print(f"El precio sugerido a vender es: {PrecioSugerido:.4f}")

    return PrecioMaximo, PrecioSugerido

In [7]:
def MainElas(UltimaTar= 1163.79):
    CondIni={
    'Dias_Anticipacion':0, 
    'Mes_Viaje':12, 
    'Fin_Semana_Viaje': 0, 
    'Buen_dia': 1
    } # La tarifa más cara de diciembre
    # sirve para estimar el precio del año 2025
    Frame=GetData(2024)
    Frame=GetFeatures(Frame)
    Coef, Elasticidad= GetElasticity(Frame)
    PrecioMaximo, PrecioSugerido= GetPrizes(Coef,CondIni,UltimaTar)

    return PrecioMaximo, PrecioSugerido, Elasticidad

In [8]:
MainElas()

Memoria usada antes: 612.76 MB
Memoria usada después: 119.22 MB
Reducción: 80.5%
El coeficiente beta1 es: -0.0275
La elasticidad de la demanda es: -0.9158
El precio maximo a vender es: 1091.5052
El precio sugerido a vender es: 959.9750


(1091.505172909186, 959.9749535505522, -0.9158051447744282)